In [51]:

import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [3]:

# Download the json from https://github.com/oganm/dnddata

url = "https://raw.githubusercontent.com/oganm/dnddata/master/data-raw/dnd_chars_unique.json"

with requests.get(url) as content:
    json_data = content.text

data = json.loads(json_data)

# Save the json file locally

with open("json_data.json", "w") as f:
    f.writelines(json.dumps(data, indent=2))

In [6]:
# open local json file

with open("json_data.json", "r") as f:
    data = json.load(f)

In [47]:
# make a Data Frame
data_dict = {
    "Name": [],
    "Race": [],
    "Class": [],
    "Total_lvl": [],
    "Alignment": [],
    "Skills": []
}

for char in data:
    char_name = data[char]["name"]["alias"][0]
    data_dict["Name"].append(char_name)

    char_race = data[char]["race"]["processedRace"][0]
    data_dict["Race"].append(char_race)

    # this could be done better
    class_str = ""
    for cls in data[char]["class"]:
        if class_str != "":
            class_str = class_str + ", "
        this_class = data[char]["class"][cls]["class"][0]
        subclass = data[char]["class"][cls]["subclass"][0]
        if subclass == "":
            subclass = "No Sub"
        class_lvl = data[char]["class"][cls]["level"][0]
        this_class_str = f"[{this_class}, {subclass}, {class_lvl}]"
        class_str = class_str + this_class_str
    data_dict["Class"].append(class_str)

    total_lvl = data[char]["level"][0]
    data_dict["Total_lvl"].append(total_lvl)
    
    if data[char]["alignment"]["processedAlignment"][0]:
        alignment = data[char]["alignment"]["processedAlignment"][0]
    else:
        alignment = None
    data_dict["Alignment"].append(alignment)

    skill_str = ""
    for skill in data[char]["skills"]:
        if skill_str != "":
            skill_str = skill_str + ", "
        skill_str = skill_str + skill
    data_dict["Skills"].append(skill_str)


df = pd.DataFrame(data=data_dict)